In [3]:
import time

try:
    import subprocess
    from datetime import datetime
    import pandas as pd
    from pytrends.request import TrendReq
    import statistics
    import csv
    from csv import reader
    import threading
    import concurrent.futures
    import copy
    import os
    import winsound
    from nordvpn_switcher import initialize_VPN,rotate_VPN,terminate_VPN
except Exception as e:
    print(str(e))
    time.sleep(30)

    
class GoogleTrend:
    '''Holds data about the stock ticker symbol,
    change in the trend/price, current trend, previous trend,
    current price, previous price, number of top occurrences
    and reason for adding it to the list'''

    def __init__(self, tickerSymbol, trendPriceRelationship = None, 
                  currentTrend = None, previousTrend = None, currentPrice = None, 
                  previousPrice = None, topOccurrences = 0, reason = None, timeScanned = None):
        '''Initializing variables'''
        self.tickerSymbol = tickerSymbol
        self.trendPriceRelationship = trendPriceRelationship
        self.currentTrend = currentTrend
        self.previousTrend = previousTrend
        self.currentPrice = currentPrice
        self.previousPrice = previousPrice
        self.topOccurrences = topOccurrences
        self.reason = reason
        self.timeScanned = timeScanned
        
    def getTickerSymbol(self):
        '''Returns the ticker symbol'''
        return self.tickerSymbol
    
    def getTrendPriceRelationship(self):
        '''Returns the change in the trend
        over the change in the stock's price'''
        
        if self.currentTrend != 0 and self.previousTrend != 0 and self.previousPrice != 0 and self.currentPrice != 0:
            # incase there is no past trend or past price
            try:
                changeInTrend = float(self.currentTrend - self.previousTrend)
                changeInPrice = float(self.currentPrice - self.previousPrice)

                # update the changeInPrice to 0.001 
                # if it is equal to zero
                if changeInPrice == 0:
                    changeInPrice = 0.001

                return abs(round(changeInTrend/changeInPrice,3))

            except:

                return 'N/A'
        else:
            return 'N/A'
    
    def getCurrentTrend(self):
        '''Returns the current google trend rating'''
        return self.currentTrend
    
    def getPreviousTrend(self):
        '''Returns the previous google trend rating'''
        return self.previousTrend
    
    def getCurrentPrice(self):
        '''Returns the current price of the stock'''
        return self.currentPrice
    
    def getPreviousPrice(self):
        '''Returns the previous price of the stock'''
        return self.previousPrice
    
    def getTopOccurrences(self):
        '''Returns the number of top ocurrences'''
        return self.topOccurrences
    
    def getReason(self):
        '''Returns the reason for adding the stock to the list'''
        return self.reason
    
    def getTimeScanned(self):
        '''Returns the time the stock was scanned'''
        return self.timeScanned

    def setTrendPriceRelationship(self, relationship):
        '''Updates the change in the trend
        over the change in the stock's price'''
        self.trendPriceRelationship = relationship
        
    def setCurrentTrend(self, currentTrend):
        '''Updates the current google trend rating'''
        self.currentTrend = currentTrend
    
    def setPreviousTrend(self, previousTrend):
        '''Updates the previous google trend rating'''
        self.previousTrend = previousTrend
    
    def setCurrentPrice(self, currentPrice):
        '''Updates the current price of the stock'''
        self.currentPrice = currentPrice
    
    def setPreviousPrice(self, previousPrice):
        '''Updates the previous price of the stock'''
        self.previousPrice = previousPrice
        
    def setReason(self, reason):
        '''Sets the reason for adding the stock to the list'''
        self.reason = reason
        
    def setTimeScanned(self,time):
        '''Updates the current time the stock was scanned'''
        self.timeScanned = time
        
    def incrementTopOccurrences(self):
        '''Updates occurrence to a boolean value'''
        self.topOccurrences += 1
        
    def isTopOccurrence(self):
        '''Returns boolean value if top occurrence'''
        return self.topOccurrence
    
# TD AMERITRADE API FUNCTIONS
# logging in

def loginToTDAmeritrade():
    # TD Ameritrade's custom API # pip install td-ameritrade-python-api
    from td.client import TDClient

    # Create a new instance of the client 
    global td_client
    td_client = TDClient(client_id = '90YQDEZ5UFQYSTQWAHMAOO4IBOE0YOEP', redirect_uri = 'https://localhost', credentials_path = r'C:\Users\hgrop\Desktop\My Python Projects\GoogleTrendsStockAlerts\GoogleTrendsV5\Untitled-1')

    # Login to a new session
    # Do this later in the loop so there is no time out
    td_client.login()

# Finds the current ask price for the stock
def askPrice(stock): 
    data = (td_client.get_quotes(instruments = [stock]))
    data = (str(data).replace(':', ','))
    data = (data.split(','))
    index = 0
    for datas in data:
        if (" 'askPrice'" == datas):
            return float(data[index + 1])
        index += 1
    
def collectingStockTickers():
    
    print("Initializing program...")
    
    # Initializes a list of tickers we want to further
    # look into 
    tickersToSearch = []

    # open the csv file that contains all the ticker in the world
    with open('ALLSTOCKS.csv', 'r') as file:

        # read all the rows and store them in the rows variable
        rows = reader(file)

        # sets the counter to 0
        counter = 0

        # starts iterating through the rows
        for row in rows:

            # increments counter by one
            counter += 1

            # Assigned the proper index to the proper cloumn
            # values used when iterating through the rest of the rows
            # only runs to read the header
            if counter == 1:
                tickerIndex = row.index('Symbol')
                nameIndex = row.index('Name')
                priceIndex = row.index('Last Sale')
                percentChangeIndex = row.index('% Change')
                marketCapIndex = row.index('Market Cap')
                volumeIndex = row.index('Volume')

            # only runs on the second iteration 
            # the row before the headers
            if counter > 1:

                # in case we encounter any companies with
                # missing data (we ignore them)
                try:

                    '''CONDITION 1'''
                    # if market cap is less than one billion (small cap) 
                    # that's a one and 9 zeros
                    marketCap = float(row[marketCapIndex])
                    if marketCap < 500000000 and marketCap > 1000:

                        '''CONDITION 2'''
                        # checks to see if the previous day's price action
                        # was not a pump and dump scheme
                        # also checks for activity
                        percentChange = abs(float(row[percentChangeIndex].replace('%','')))
                        if percentChange < 400:

                            '''CONDITION 3'''
                            # checks price to make sure it is less than $8
                            price = float(row[priceIndex].replace('$',''))
                            if price < 10 and price > 0:

                                '''FINAL CONDITION'''
                                # if all conditions are met
                                # and it is not a derivative
                                ticker = row[tickerIndex]
                                if ticker.isalpha():
                                    # adds ticker to the list
                                    # of tickers to searchs
                                    tickersToSearch.append(ticker)

                except:
                    pass
    
    print("Program initialized.")
    print("Found " + str(len(tickersToSearch)) + " tickers!\n")
    
    return tickersToSearch

def mean(data):
    '''Finds the mean of a list'''
    if len(data) > 1:
        
        sum = 0
        
        for element in data:
            sum += element
            
        return float(round((sum / len(data)),2))
    
    else:
        return data[0]

def splitList(dataList):
    '''Splits a list into two separate parts'''
    
    if len(dataList) > 1:
        
        # initialize variables
        firstHalf = []
        secondHalf = []
        counter = 0

        # if the length of the list is odd
        if len(dataList) % 2 == 1:
            dataList.pop(0)
            
        halfWayMark = int(len(dataList) / 2)
            
        return dataList[:halfWayMark], dataList[halfWayMark:]
    
def trendSearch(tickerToSearch, pytrends):

    # creating payload
    pytrends.build_payload([(tickerToSearch + " stock")], cat=0, timeframe = 'now 1-d')
    data = pytrends.interest_over_time()
    
    # converting trends data to a readable list
    data = data.values.tolist()
    
    return data, tickerToSearch

def dataCleanup(data):
    '''Cleans the data for later use'''
    
    # Initializing an empty google data trends list
    googleTrendsData = []

    # iterating through the google trends data
    for popularityScore in data:
        
        googleTrendsData.append(popularityScore[0])
    
    return googleTrendsData
    
def splitListIntoSections(data, sections):
    '''Splits a list into groups of 10s'''
    
    newList = []
    tempList = []
    
    for element in data:
        
        tempList.append(element)
        
        if len(tempList) >= sections:
            newList.append(tempList)
            tempList = []        
    
    if len(tempList) > 0:
        newList.append(tempList)
        
    return newList

def sortData():
    '''Sort stock data from highest
    variance to the least variance'''
    
    blacklist = ['RAM','ASM','BEST','ONE','UK','FORD','JOB','CAP',
                 'FORD','VERY','LIFE','AGE','GAME','TOUR','EARN','LOAN',
                 'NBA','VS','AUTO','NEW']
    
    unsortedData = []
    sortedData = []
    copyOfStockData = copy.deepcopy(stockData)

    for stock in stockData:
        unsortedData.append(stock.getCurrentTrend())

    while len(unsortedData) != 0:
        maximum = 0
        index = -1
        for data in unsortedData:
            index += 1
            if data >= maximum:
                maximumIndex = index
                maximum = data
                
        sortedData.append(copyOfStockData[maximumIndex])
        unsortedData.pop(maximumIndex)
        copyOfStockData.pop(maximumIndex)
        

    return sortedData

def createObjects(innerElement):
    '''Searches the stock ticker and filters out bad data'''
    
    global stockNames
    
    try:
        # Getting data
        data, ticker = trendSearch(innerElement, TrendReq(hl='en-US', tz=360))
    except:
        return 'error'

    # getting the current time
    theTime = (currentTime()[0] + ":" + currentTime()[1])
    
    # incase there is no data (try block)
    try:
        # cleans data and removes unwanted columns
        cleanData = dataCleanup(data)
        
        # calculates mean of the daya
        meanOfData = mean(cleanData)
        
        # calculates the current trend over the mean of the entire data set
        result = round(cleanData[-1]/meanOfData,3)
        
        # gets the number of spikes in the data
        count30, count60, count90, count100 = catchChoppyData(cleanData)
        
        # determines if the data is worthy
        if result <= 1:
            reason = 'low result'
            result = 0
        elif cleanData.count(0) >= int(len(cleanData) - 3):
            reason = 'too many zeros'
            result = 0
        elif (count30 >= 15 or count60 >= 3 or count90 >= 2 or count100 == 1):
            reason = 'spiked data'
            result = 0
        elif cleanData[len(cleanData) - 5:].count(0) >= 2:
            reason = 'front zeros'
            result = 0
        else:
            reason = 'good!'
        
        
        
        if innerElement not in stockNames: 
            stockData.append(GoogleTrend(tickerSymbol = ticker, currentTrend = result, 
                                            reason = reason , timeScanned = theTime))
        # output to the console
        print("Scanned: " + ticker + " // Trend: " + str(result))
        
        stockNames.add(innerElement)
        
        
    except:
        # if there is no data then execute the following
        reason = 'no data'
        result = 0
        if innerElement not in stockNames:
            stockData.append(GoogleTrend(tickerSymbol = ticker, currentTrend = result, 
                                            reason = reason , timeScanned = theTime))
        stockNames.add(innerElement)
        
def modifyObjects(innerElement):
    '''Searches the stock ticker and filters out bad data then
    modifies the objects in the stockData list'''
    
    global stockData
    global td_client
    
    # finding index to modify
    index = -1
    
    # gets the index
    for stock in stockData:
        if innerElement == stockData[index].getTickerSymbol():
            break
        else:
            index += 1
            
    # if top occurrence
    if stockData[index].getTickerSymbol() in tickers[:15]:
        stockData[index].incrementTopOccurrences()

    try:
        # Getting data
        data, ticker = trendSearch(innerElement, TrendReq(hl='en-US', tz=360))
    except Exception as e:
        print(str(e))
        return 'error'

    # getting the current time
    theTime = (currentTime()[0] + ":" + currentTime()[1])
    
    # if it is a top occurrence check the price 
    # using the TD ameritrade API
    if stockData[index].getTopOccurrences() >= 2:
        stockData[index].setPreviousPrice(stockData[index].getCurrentPrice())
        stockData[index].setCurrentPrice(askPrice(stockData[index].getTickerSymbol()))
        
    # incase there is no data (try block)
    try:
        # cleans data and removes unwanted columns
        cleanData = dataCleanup(data)
        
        # calculates mean of the daya
        meanOfData = mean(cleanData)
        
        # calculates the current trend over the mean of the entire data set
        result = round(cleanData[-1]/meanOfData,3)
        
        # gets the number of spikes in the data
        count30, count60, count90, count100 = catchChoppyData(cleanData)
        
        # determines if the data is worthy
        if result <= 1:
            reason = 'low result'
            result = 0
        elif cleanData.count(0) >= int(len(cleanData) - 3):
            reason = 'too many zeros'
            result = 0
        elif (count30 >= 15 or count60 >= 3 or count90 >= 2 or count100 == 1):
            reason = 'spiked data'
            result = 0
        elif cleanData[len(cleanData) - 5:].count(0) >= 2 :
            reason = 'front zeros'
            result = 0
        else:
            reason = 'good!'
        
        # modifying object
        stockData[index].setPreviousTrend(stockData[index].getCurrentTrend())
        stockData[index].setCurrentTrend(result)
        stockData[index].setReason(reason)
        stockData[index].setTimeScanned(theTime)
        stockData[index].setTrendPriceRelationship(stockData[index].getTrendPriceRelationship())
        
        # output to the console
        print("Scanned: " + ticker + " // Trend: " + str(result))
        
        
    except:
        # if there is no data then execute the following
        reason = 'no data'
        result = 0
        
        # modifying object
        stockData[index].setPreviousTrend(stockData[index].getCurrentTrend())
        stockData[index].setCurrentTrend(result)
        stockData[index].setReason(reason)
        stockData[index].setTimeScanned(theTime)
        stockData[index].setTrendPriceRelationship(stockData[index].getTrendPriceRelationship())

def currentTime():
    '''Returns current time 
    hour, minute, second'''

    # datetime object containing current date and time
    now = datetime.now()
    
    todaysDate = str(now).split(" ")[0]
    parsedData = str(now).split(" ")[1].split(':')
    hour = parsedData[0]
    minute = parsedData[1]
    second = parsedData[2]
    militaryTime = (str(hour) + ':' + str(minute))

    return hour, minute, second, militaryTime, todaysDate

def findMinimumIndex(dataList):
    '''Finds a lists minimum data point index'''
    
    minimum = 100
    
    for data in dataList:
        
        if data < minimum:
            minimum = data
            
            
    return dataList.index(minimum), minimum
        
def catchChoppyData(dataList):
    '''Catches outliers and spikes inside data sets'''
    
    index = -1
    count30 = 0
    count60 = 0
    count90 = 0
    count100 = 0
    
    for _ in dataList:
        index += 1
        
        if dataList[index] + 30 <= dataList[index + 1]:
            count30 += 1
        if dataList[index] + 60 <= dataList[index + 1]:
            count60 += 1
        if dataList[index] + 90 <= dataList[index + 1]:
            count90 += 1
        if dataList[index] + 100 <= dataList[index + 1]:
            count100 += 1
            
        if index == len(dataList) - 2:
            return count30,count60,count90,count100
    
def displayData():
    '''Collects data from the stockData
    and creates a panda data frame from it'''
    
    stockData = sortData()
    
    df = {}

    valList = []
    for stock in stockData:
        valList.append(list(vars(stock).values()))

    columnHeaders = list(vars(stockData[0]).keys())
    for header in columnHeaders:
        firstItter = True
        for val in valList:
            if firstItter:
                df[header] = [val[columnHeaders.index(header)]]
                firstItter = False
            else:
                df[header] += [val[columnHeaders.index(header)]]
                
    return pd.DataFrame(df)
            

def main():
    '''Executes the whole program'''
    
    global stockDataFrame
    global stockData
    global tickers
    global td_client
    global tickers
    
    # starts the timer
    start = time.perf_counter()
    
    # keeps track of the number of scans (iterations)
    iteration = 0
    
    # current timeframe being used in google trends
    print("Using timeframe (" + 'now 1-d' + ")\n")
        
    # terminates VPN if cnnected
    try:
        terminate_VPN()
        print("...From the current VPN connection...\n")
        time.sleep(3)
    except:
        pass
    
    # After terminating VPN login to the TD ameritrade API client
    # do this everytime so there is no timeout
    loginToTDAmeritrade()
    
    # initializes VPN settings
    settings = initialize_VPN(save=1,area_input=['Canada','united states'])
    
    # rotate VPN
    rotate_VPN(settings)
        
    # collects tickers based off constraints
    tickers = collectingStockTickers()
    
    # if the stock data is already populated
    # then make a new tickers list with the 
    # most popular stocks in the front
    if len(stockData) == len(tickers):
        print("Multiple runs... modifying objects")
        tickers = copy.deepcopy([])
        for stock in sortData():
            tickers.append(stock.getTickerSymbol())
    
    # splits data into lists of 8 elements (I have possible 8 threads)
    dataList = splitListIntoSections(tickers, 8)
    
    # iterates through the split lists inside the "big" list
    for outterList in dataList:

        # if there is a BITCH error
        bitch = 0
        
        
        # if stock data is populated
        if len(stockData) == len(tickers):
            with concurrent.futures.ThreadPoolExecutor() as executor:
                dataToAdd = executor.map(modifyObjects,outterList)
        else:
            # in sets of specified sections
            with concurrent.futures.ThreadPoolExecutor() as executor:
                dataToAdd = executor.map(createObjects,outterList)

        # error handling
        for e in dataToAdd:
            if e == 'error' and bitch == 0:
                bitch = 1
                rotate_VPN(settings) 
                dataList.append(outterList)
                time.sleep(0.5)
                
        # every 3 scans pass the stockData to the kernel to be displayed
        # passes variable along to the kernel, so it can be used in
        # other notebook simultaneously
        iteration += 1
        if iteration % 3 == 0:
            stockDataFrame = displayData()
            %store stockDataFrame
        
            
            
            
    # one last time after loop
    stockDataFrame = displayData()
    %store stockDataFrame
            
    # Gets the current date and time
    timeStamp = ("-" + currentTime()[4] + "@" + currentTime()[3]).replace(':',"-")
    
    # gets the ending time
    end = time.perf_counter()
    
    # calculates the time in minutes from the start to end
    totalTimeInMinutes = round(((end - start)/60),2)
    
    # writing data to csv
    csvFileName = ('GoogleTrends' + timeStamp)
    stockDataFrame.to_csv(r'C:\Users\hgrop\Desktop\My Python Projects\GoogleTrendsStockAlerts\GoogleTrendsV5\ ' + str(csvFileName) + '.csv', index = False, header=True)
    
    # outputs to file
    print("\nThe program took " + str(totalTimeInMinutes) + " minutes to run.")
    
    # Turn the VPN off after completed scan 
    terminate_VPN()
    time.sleep(3)
            
    # short delay in between runs
    time.sleep(5)

ERROR! Session/line number was not unique in database. History logging moved to new session 1232


In [2]:
stockData = []
stockNames = set()

In [ ]:
while True:
    try:
        main()
        time.sleep(10)
    except Exception as e:
        print(str(e))

Using timeframe (now 1-d)

Trying to load saved settings...
Saved settings loaded!


Disconnecting...
Done!
...From the current VPN connection...

You're using Windows.
Performing system check...
###########################

NordVPN installation check: ✓
NordVPN service check: ✓
Opening NordVPN app and disconnecting if necessary...
NordVPN app launched: ✓
#####################################

You've entered a list of connection options. Checking list...


Saving settings in project folder...


Done!


Your current ip-address is: 69.136.101.196

Connecting you to united states ...

your new ip-address is: 83.136.182.185

Done! Enjoy your new server.

Initializing program...
Program initialized.
Found 1736 tickers!

Multiple runs... modifying objects
Scanned: VSTM // Trend: 0
Scanned: THMO // Trend: 8.214
Scanned: DYAI // Trend: 4.684
Scanned: ATER // Trend: 1.283
Scanned: LYRA // Trend: 16.136
Scanned: LITB // Trend: 23.981
Scanned: BRQS // Trend: 1.418
Scanned: TACO // Trend: 0
Scanne

Stored 'stockDataFrame' (DataFrame)
Scanned: TATT // Trend: 0
Scanned: TARA // Trend: 0
Scanned: TAST // Trend: 0
Scanned: TANH // Trend: 0
Scanned: SYN // Trend: 0
Scanned: TAOP // Trend: 0
Scanned: TC // Trend: 1.681
Scanned: SYRS // Trend: 0
Scanned: SWBK // Trend: 0
Scanned: SYBX // Trend: 0
Scanned: SYPR // Trend: 0
Scanned: SYTA // Trend: 0
Scanned: SWET // Trend: 0
Scanned: TAIT // Trend: 0
Scanned: SVT // Trend: 0
Scanned: SXTC // Trend: 0
Scanned: SVRA // Trend: 0
Scanned: SVOK // Trend: 0
Scanned: SUNW // Trend: 0
Scanned: SWZ // Trend: 0
Stored 'stockDataFrame' (DataFrame)
Scanned: SUPV // Trend: 0
Scanned: SURF // Trend: 0
Scanned: SUP // Trend: 0
Scanned: SVFD // Trend: 0
Scanned: STPC // Trend: 0
Scanned: STWO // Trend: 0
Scanned: STRM // Trend: 0
Scanned: SSKN // Trend: 0
Scanned: STSA // Trend: 0
Scanned: STIM // Trend: 0
Scanned: STON // Trend: 0
Scanned: STKS // Trend: 0
Scanned: SSY // Trend: 0
Scanned: STRR // Trend: 0
Scanned: SSNT // Trend: 0
Scanned: SRTS // Tren

The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
Scanned: PCSA // Trend: 0
Scanned: PCTI // Trend: 0

Your current ip-address is: 37.19.212.143

Connecting you to united states ...

your new ip-address is: 185.207.249.67

Done! Enjoy your new server.

The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with c

Stored 'stockDataFrame' (DataFrame)
Scanned: LEJU // Trend: 0
Scanned: LEDS // Trend: 0
Scanned: KZR // Trend: 0
Scanned: LBPS // Trend: 0
Scanned: LAIX // Trend: 0
Scanned: KURI // Trend: 0
Scanned: KUKE // Trend: 0
Scanned: KRMD // Trend: 0
Scanned: KULR // Trend: 0
Scanned: KVSA // Trend: 0
Scanned: KXIN // Trend: 0
Scanned: KZIA // Trend: 0
Scanned: KTRA // Trend: 0
Scanned: KNDI // Trend: 0
Scanned: KRBP // Trend: 0
Scanned: KSI // Trend: 0
Scanned: KRNL // Trend: 0
Scanned: KSMT // Trend: 0
Scanned: KTCC // Trend: 0
Scanned: KMPH // Trend: 11.947
Stored 'stockDataFrame' (DataFrame)
Scanned: KLXE // Trend: 0
Scanned: KMF // Trend: 0
Scanned: KMDA // Trend: 0
Scanned: KOR // Trend: 0
Scanned: KBNT // Trend: 0
Scanned: KFS // Trend: 0
Scanned: KFFB // Trend: 0
Scanned: KRKR // Trend: 0
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
Scanned: KERN 

Scanned: GAME // Trend: 1.536

Your current ip-address is: 37.19.213.148

Connecting you to canada ...

your new ip-address is: 162.253.71.39

Done! Enjoy your new server.

Stored 'stockDataFrame' (DataFrame)
Scanned: FUND // Trend: 1.591
Scanned: FVT // Trend: 0
Scanned: FVE // Trend: 0
Scanned: FWP // Trend: 0
Scanned: FURY // Trend: 0
Scanned: FUSN // Trend: 0
Scanned: FTK // Trend: 0
Scanned: FULC // Trend: 1.132
Scanned: FTF // Trend: 0
Scanned: FTEK // Trend: 0
Scanned: FTFT // Trend: 0
Scanned: FUSE // Trend: 0
Scanned: FSEA // Trend: 0
Scanned: FTRP // Trend: 0
Scanned: FT // Trend: 4.256
Scanned: FSRX // Trend: 0
Scanned: FSII // Trend: 0
Scanned: FSI // Trend: 0
Stored 'stockDataFrame' (DataFrame)
Scanned: FRBK // Trend: 0
Scanned: FRON // Trend: 0
Scanned: FSTX // Trend: 0
Scanned: FSSI // Trend: 0
Scanned: FRSX // Trend: 0
Scanned: FREQ // Trend: 0
Scanned: FNHC // Trend: 0
Scanned: FNCB // Trend: 0
Scanned: FRLN // Trend: 0
Scanned: FORD // Trend: 1.959
Scanned: FPAY // Tr

...

Stored 'stockDataFrame' (DataFrame)
Scanned: NHTC // Trend: 0
Scanned: NGL // Trend: 0
Scanned: NERV // Trend: 0
Scanned: NINE // Trend: 0
Scanned: NH // Trend: 0
Scanned: NGCA // Trend: 0
Scanned: NL // Trend: 0
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
Scanned: NES // Trend: 0
Scanned: NGAC // Trend: 0
Scanned: NEPT // Trend: 0

Your current ip-address is: 37.19.212.34

Connecting you to canada ...

your new ip-address is: 37.19.212.115

Done! Enjoy your new server.

Scanned: NBRV // Trend: 0
Scanned: NBSE // Trend: 0
Scanned: NAOV // Trend: 0
Scanned: NAK // Trend: 0
Scanned: NAVB // Trend: 0
Scanned: NCZ // Trend: 0
Scanned: NAT // Trend: 0
Scanned: NBEV // Trend: 0
Stored 'stockDataFrame' (DataFrame)
Scanned: MTR // T

Scanned: HTBX // Trend: 0
Scanned: HRTG // Trend: 0
Scanned: HUIZ // Trend: 0
Scanned: HT // Trend: 0
Scanned: HUGE // Trend: 0
Scanned: HSTO // Trend: 0
Scanned: HROW // Trend: 0
Scanned: HOTH // Trend: 0
Scanned: HNRG // Trend: 0
Scanned: HOOK // Trend: 0
Scanned: HTGM // Trend: 0
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
Scanned: HNNA // Trend: 0
Scanned: HMLP // Trend: 0
Scanned: HITI // Trend: 0
Scanned: HJLI // Trend: 0

Your current ip-address is: 185.207.249.145

Connecting you to canada ...

your new ip-address is: 37.19.212.74

Done! Enjoy your new server.

Stored 'stockDataFrame' (DataFrame)
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
Scanned: HIO // Trend: 0
Scanned: HCWB // Trend: 0
Scanned: HIGA // Trend: 0
Scanned: HIL // Trend: 0
Scanned: HIX // Trend: 0
Scanned: HIHO // Trend: 0

Your current ip-address is: 37

Scanned: EEX // Trend: 0
Scanned: EIGR // Trend: 0
Scanned: EKSO // Trend: 0
Scanned: EEIQ // Trend: 0
Scanned: EDN // Trend: 0
Scanned: EJH // Trend: 1.23
Scanned: EDAP // Trend: 0
Scanned: EDSA // Trend: 0
Scanned: EFOI // Trend: 0
Scanned: EDTK // Trend: 0
Scanned: DYNT // Trend: 0
Scanned: EDF // Trend: 0
Scanned: EBON // Trend: 0
Scanned: EFL // Trend: 0
Scanned: EDI // Trend: 0
Scanned: DVD // Trend: 0
Scanned: DXF // Trend: 0
Scanned: DXYN // Trend: 0
Scanned: ECOR // Trend: 0
Scanned: DWSN // Trend: 0Scanned: EDD // Trend: 0

Stored 'stockDataFrame' (DataFrame)
Scanned: DYNS // Trend: 0
Scanned: DTEA // Trend: 0
Scanned: DUO // Trend: 0
Scanned: DUOT // Trend: 0
Scanned: DTST // Trend: 0
Scanned: DSX // Trend: 0
Scanned: DRTT // Trend: 0
Scanned: DS // Trend: 0
Scanned: DSS // Trend: 0
Scanned: DSM // Trend: 0
Scanned: DPW // Trend: 0
Scanned: DSAC // Trend: 0
Scanned: DTSS // Trend: 0
Scanned: DOMA // Trend: 0
Scanned: DPRO // Trend: 0
Scanned: DNAC // Trend: 0
Scanned: DNAA /

Scanned: AVCO // Trend: 0
Stored 'stockDataFrame' (DataFrame)
Scanned: AUUD // Trend: 0
Scanned: AUD // Trend: 0
Scanned: ATXI // Trend: 0
Scanned: AUMN // Trend: 0
Scanned: AVCT // Trend: 0
Scanned: AVDL // Trend: 0
Scanned: AUTL // Trend: 0
Scanned: ATHE // Trend: 0
Scanned: ATHA // Trend: 0
Scanned: ATNX // Trend: 0
Scanned: ATHX // Trend: 0
Scanned: ATNF // Trend: 6.477
Scanned: ATIF // Trend: 0
Scanned: ASUR // Trend: 0
Scanned: ASTC // Trend: 0
Scanned: ASRV // Trend: 0
Scanned: ASYS // Trend: 0
Scanned: ATAX // Trend: 0
Scanned: ATA // Trend: 0
Scanned: ASRT // Trend: 0
Stored 'stockDataFrame' (DataFrame)
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
Scanned: ASLN // Trend: 0
Scanned: ASPS // Trend: 0
Scanned: ASM // Tr


Your current ip-address is: 83.136.182.80

Connecting you to canada ...

your new ip-address is: 37.19.212.98

Done! Enjoy your new server.

The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.

Your current ip-address is: 37.19.212.98

Connecting you to canada ...

your new ip-address is: 37.19.212.194

Done! Enjoy your new server.

The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returned a response with code 429.
The request failed: Google returne